# Notebook for own Data

In [1]:
import os
import gc
#import pytz
import operator
import numpy as np
import pickle as pkl
import xgboost as xgb
from time import sleep
from datetime import datetime
from sklearn.preprocessing import StandardScaler
import datasets

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action = 'ignore', category = FutureWarning)
warnings.filterwarnings(action = 'ignore', category = DeprecationWarning)

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import sys
sys.path.append('..')

#time = datetime.now(pytz.timezone('Europe/Oslo')).strftime('%m.%d.%Y_%H.%M.%S')
#print(f'Notebook initialized execution at {time}.')

ModuleNotFoundError: No module named 'xgboost'

## General Methods

In [ ]:
def memory_optimization(dfs):
    for df in dfs:
        del df
    gc.collect()

## XGB Training

In [ ]:
def xgb_train( train_X, train_y, validation_X, validation_y):
    model_name_wrt = f'../models/model_finn.hdf5'

    xgb_model = xgb.XGBRegressor(base_score = 0.5, booster = 'gbtree', colsample_bylevel = 1,
                                 colsample_bytree = 1, gamma = 0, importance_type = 'gain',
                                 learning_rate = 0.1, max_delta_step = 0, max_depth = 9,
                                 min_child_weight = 1, missing = None, n_estimators = 10000, n_jobs = -1,
                                 nthread = None, objective = 'reg:squarederror', random_state = 101, reg_alpha = 2,
                                 reg_lambda = 0.2, scale_pos_weight = 1, seed = None, silent = False, subsample = 1)

    xgb_model.fit(train_X, train_y, eval_set = [(validation_X, validation_y)], eval_metric = 'mae', 
                  early_stopping_rounds = 32, verbose = True)   
    
    xgb_model.save_model(model_name_wrt)
    
    return xgb_model

In [ ]:
def importance(xgb_model, train_X):
    input_features = train_X.columns.values
    feat_imp = xgb_model.feature_importances_
    np.split(feat_imp, len(input_features))
    
    feat_imp_dict = {}
    for i in range(0, len(input_features)):
        feat_imp_dict[feat_imp[i]] = input_features[i]

    sorted_feats = sorted(feat_imp_dict.items(), key = operator.itemgetter(0))
    for i in range(len(sorted_feats) - 1, 0, -1):
        print(sorted_feats[i])

## Prepare Data

In [ ]:
start_time = datetime.now()

train_X, train_y, validation_X, validation_y, test_X, test_y = datasets.load(f'../input/finn.csv')

## Train and predict

In [ ]:
xgb_model = xgb_train(train_X, train_y, validation_X, validation_y)   

    

## Test

In [ ]:
df = pd.DataFrame()
df['pred'] = xgb_model.predict(test_X)
df['target'] = test_y.reset_index(drop=True)
df['difference'] = df['pred'] - df['target']
df['difference %'] = (df['pred'] / df['target'] - 1) * 100

print(df)